In [44]:
import psycopg2
import pandas as pd
import psycopg2
from IPython.display import display

host = "silly.db.elephantsql.com"
port = "5432"
database = "ljppladj"
user = "ljppladj"
password = "JFgtpa3EjnB6zOJ75ejGn_gUOl8-dy3j"

try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar: {e}")


Conexão bem-sucedida!


In [45]:
# import psycopg2

# Crie um cursor para executar comandos SQL no banco de dados
cursor = conn.cursor()

# Execute a consulta SQL para juntar as tabelas e obter os dados desejados
cursor.execute("""
    SELECT 
        candidates.cnd_id, 
        candidates.cnd_age, 
        candidates.cnd_name, 
        candidates.cnd_current_profession, 
        candidates.cnd_experience, 
        string_agg(DISTINCT skl_skill.skl_name, ',') as skills, 
        string_agg(DISTINCT skl_skill.skl_description, ',') as skill_descriptions, 
        string_agg(DISTINCT knw_knowledge.knw_name, ',') as knowledge_names, 
        string_agg(DISTINCT knw_knowledge.knw_description, ',') as knowledge_descriptions, 
        string_agg(DISTINCT att_attitude.att_name, ',') as attitude_names, 
        string_agg(DISTINCT att_attitude.att_description, ',') as attitude_descriptions
    FROM candidates
    INNER JOIN candidate_skill ON candidates.cnd_id = candidate_skill.cnd_id
    INNER JOIN skl_skill ON candidate_skill.knw_id = skl_skill.skl_id
    INNER JOIN knw_knowledge ON skl_skill.skl_id = knw_knowledge.knw_id
    INNER JOIN candidate_attitude ON candidates.cnd_id = candidate_attitude.cnd_id
    INNER JOIN att_attitude ON candidate_attitude.knw_id = att_attitude.att_id
    GROUP BY candidates.cnd_id, candidates.cnd_age, candidates.cnd_name, candidates.cnd_current_profession, candidates.cnd_experience
""")


dfCandidatos = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(dfCandidatos)


cursor.close()

,cnd_id,cnd_age,cnd_name,cnd_current_profession,cnd_experience,skills,skill_descriptions,knowledge_names,knowledge_descriptions,attitude_names,attitude_descriptions
0,1,25,João,Estagiário,0,"Desenvolvimento de Aplicações Simples,Programa...",Capacidade de criar aplicações web simples com...,"Design Patterns,Testes Automatizados",Prática de escrever códigos para testar automa...,"Aprendizado Contínuo,Atenção ao Cliente Intern...",Capacidade de analisar problemas complexos e d...
1,2,23,Maria Oliveira,Desenvolvedor Júnior,0,"Desenvolvimento de Aplicações Simples,Programa...",Capacidade de criar aplicações web simples com...,"Design Patterns,Testes Automatizados",Prática de escrever códigos para testar automa...,"Aprendizado Contínuo,Atenção ao Cliente Intern...",Capacidade de analisar problemas complexos e d...
2,3,24,Rafael Pereira,Estagiário,0,"Desenvolvimento de Aplicações Simples,Programa...",Capacidade de criar aplicações web simples com...,"Design Patterns,Testes Automatizados",Prática de escrever códigos para testar automa...,"Aprendizado Contínuo,Atenção ao Cliente Intern...",Capacidade de analisar problemas complexos e d...
3,4,22,Luana Santos,Estagiária,0,"Desenvolvimento de Aplicações Simples,Programa...",Capacidade de criar aplicações web simples com...,"Design Patterns,Testes Automatizados",Prática de escrever códigos para testar automa...,"Aprendizado Contínuo,Atenção ao Cliente Intern...",Capacidade de analisar problemas complexos e d...
4,5,23,Felipe Oliveira,Desenvolvedor Júnior,0,"Desenvolvimento de Aplicações Simples,Programa...",Capacidade de criar aplicações web simples com...,"Design Patterns,Testes Automatizados",Prática de escrever códigos para testar automa...,"Aprendizado Contínuo,Atenção ao Cliente Intern...",Capacidade de analisar problemas complexos e d...
5,6,23,Gabriel Almeida,Desenvolvedor Júnior,0,"Desenvolvimento de Aplicações Simples,Programa...",Capacidade de criar aplicações web simples com...,"Design Patterns,Testes Automatizados",Prática de escrever códigos para testar automa...,"Aprendizado Contínuo,Atenção ao Cliente Intern...",Capacidade de analisar problemas complexos e d...


In [46]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics


desenvolvedor = {
    'profissao': 'Desenvolvedor Back-End',
    'habilidades': [
        'Python', 
        'Java', 
        'Node.js', 
        'Bancos de Dados (por exemplo, MySQL, PostgreSQL)',
        'API Design',
        'Trabalho com APIs RESTful'
    ],
    'conhecimentos': [
        'Manipulação de Dados',
        'Conhecimento em SQL',
    ],
    'atitudes': [
        'Lógica de Programação',
        'Resolução de Problemas',
        'Trabalho em Equipe',
    ]
}


habilidades = desenvolvedor['habilidades']
conhecimentos = desenvolvedor['conhecimentos']
atitudes = desenvolvedor['atitudes']

texto_habilidades = ' '.join(habilidades)
texto_conhecimentos = ' '.join(conhecimentos)
texto_atitudes = ' '.join(atitudes)

cha_vaga = f'{texto_habilidades} {texto_conhecimentos} {texto_atitudes}'

dfCandidatos['CHA'] = dfCandidatos['skills'] + ' ' + dfCandidatos['knowledge_names'] + ' ' + dfCandidatos['attitude_names']
dfCandidatos['Rotulo'] = 0

# dfCandidatos['CHA_vaga'] = texto_completo

In [57]:

# display(dfCandidatos)

X_train, X_test, y_train, y_test = train_test_split(dfCandidatos['CHA'], dfCandidatos['Rotulo'], test_size=0.2, random_state=42)
# Adicionando a coluna 'cha_vaga' ao DataFrame
dfCandidatos['cha_vaga'] = cha_vaga

# Concatenando as características dos candidatos com os requisitos da vaga
X_train = dfCandidatos['CHA'] + ' ' + dfCandidatos['cha_vaga']

# Modelo de classificação
model = make_pipeline(CountVectorizer(), MultinomialNB())
model.fit(X_train, y_train)


y_pred = model.predict(X_test)

# Obtendo as probabilidades previstas pelo modelo para a classe 'apto'
probabilidades_apto = model.predict_proba(X_test)[:, 0]

# # Adicionando as probabilidades ao DataFrame
dfCandidatos.loc[X_test.index, 'Probabilidade_Apto'] = probabilidades_apto

# # Ordenando os candidatos pela probabilidade de serem 'apto'
candidatos_ordenados = dfCandidatos.sort_values(by='Probabilidade_Apto', ascending=False)

# # Selecionando os 3 candidatos mais prováveis
top_candidatos = candidatos_ordenados.head(3)

display(top_candidatos)

0    Desenvolvimento de Aplicações Simples,Programa...
1    Desenvolvimento de Aplicações Simples,Programa...
2    Desenvolvimento de Aplicações Simples,Programa...
3    Desenvolvimento de Aplicações Simples,Programa...
4    Desenvolvimento de Aplicações Simples,Programa...
5    Desenvolvimento de Aplicações Simples,Programa...
Name: CHA, dtype: object


ValueError: Found input variables with inconsistent numbers of samples: [6, 4]